In [1]:
import pandas as pd
import numpy as np
from numpy.random import permutation
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Merge
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop

Using TensorFlow backend.


In [2]:
import tensorflow

In [1]:
# tensorflow
import tensorflow
print('tensorflow: %s' % tensorflow.__version__)
# keras
import keras
print('keras: %s' % keras.__version__)

tensorflow: 1.5.0
keras: 2.1.3


Using TensorFlow backend.


In [2]:
tensorflow.Session()

In [29]:
tenz = np.load('mi43ge.npy')
# tenz[0,i] = parse_seq_r(mi43[i],43)      reversed
# tenz[1,i] = parse_seq_r(ge43[i],43)
tenz= tenz[:,:10000]

In [30]:
tenz.shape[1]/2

5000.0

In [31]:
sh = int(tenz.shape[1]/2)
tenz0sh = tenz[:,:sh]
perm0 = permutation(sh)
tenz0perm = tenz0sh[:,perm0]
print(tenz0perm.shape)
tar0 = np.zeros(sh)
tar1 = np.ones(sh)
tar01 = np.concatenate((tar0,tar1))
permt = permutation(sh*2)
tenz01 = np.concatenate((tenz0perm,tenz[:,sh:]), axis=1)
print(tenz01.shape)
mi = tenz01[0,permt].reshape(tenz01.shape[1],tenz01.shape[2],tenz01.shape[3],1)
ge = tenz01[1,permt].reshape(tenz01.shape[1],tenz01.shape[2],tenz01.shape[3],1)
tar = tar01[permt]

(2, 5000, 43, 4)
(2, 10000, 43, 4)


In [32]:
mi.shape

(10000, 43, 4, 1)

In [41]:
genes_branch = Sequential()
genes_branch.add(Conv2D(64,(4,3), input_shape=(ge.shape[1],ge.shape[2],1), padding='same'))
#genes_branch.add(Dense(64))
genes_branch.add(Activation('relu'))
genes_branch.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
genes_branch.add(Dropout(0.5))
genes_branch.add(Dense(32))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))

mirs_branch = Sequential()
mirs_branch.add(Conv2D(64,(4,3), input_shape=(ge.shape[1],ge.shape[2],1), padding='same'))
#mirs_branch.add(Dense(64))
mirs_branch.add(Activation('relu'))
mirs_branch.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
mirs_branch.add(Dropout(0.5))
mirs_branch.add(Dense(32))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))

merged = Merge([genes_branch, mirs_branch], mode='concat')
model = Sequential()
model.add(merged)
model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))

#model.add(Dropout(0.5))
#model.add(Dense(2))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('relu')) #sigmoid

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='mean_squared_error', #'binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])


/home/kira/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [43]:
model.fit([ge, mi], tar,
              batch_size=50,
              nb_epoch=1000,
              validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/1000
 300/8000 [>.............................] - ETA: 4s - loss: 0.2533 - acc: 0.5033

/home/kira/.local/lib/python3.5/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


8000/8000 [==============================] - 4s 491us/step - loss: 0.2528 - acc: 0.5026 - val_loss: 0.2562 - val_acc: 0.4885
Epoch 2/1000
8000/8000 [==============================] - 4s 470us/step - loss: 0.2528 - acc: 0.5052 - val_loss: 0.2612 - val_acc: 0.4885
Epoch 3/1000
8000/8000 [==============================] - 4s 472us/step - loss: 0.2526 - acc: 0.5024 - val_loss: 0.2611 - val_acc: 0.4885
Epoch 4/1000
8000/8000 [==============================] - 4s 478us/step - loss: 0.2524 - acc: 0.4997 - val_loss: 0.2573 - val_acc: 0.4885
Epoch 5/1000
8000/8000 [==============================] - 4s 488us/step - loss: 0.2522 - acc: 0.4970 - val_loss: 0.2566 - val_acc: 0.4885
Epoch 6/1000
8000/8000 [==============================] - 4s 488us/step - loss: 0.2523 - acc: 0.5013 - val_loss: 0.2550 - val_acc: 0.4885
Epoch 7/1000
8000/8000 [==============================] - 4s 466us/step - loss: 0.2504 - acc: 0.5136 - val_loss: 0.2566 - val_acc: 0.4885
Epoch 8/1000
8000/8000 [=======================